In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#train = pd.read_csv("train_newpreproc.csv",index_col=0)
#val = pd.read_csv("val_newpreproc.csv",index_col=0)
#test = pd.read_csv("test_newpreproc.csv",index_col=0)
train = pd.read_csv("train_oldpreproc.csv",index_col=0)
val = pd.read_csv("val_oldpreproc.csv",index_col=0)
test = pd.read_csv("test_oldpreproc.csv",index_col=0)
trainval = pd.concat([train,val],axis=0)

### VADER

Developed in 2014, VADER (Valence Aware Dictionary and sEntiment Reasoner) is a pre-trained model that uses rule-based values tuned to sentiments from social media. It evaluates the text of a message and gives you an assessment of not just positive and negative, but the intensity of that emotion as well.

It uses a dictionary of terms that it can evaluate. From the GitHub repository this includes examples like:

Negations - a modifier that reverses the meaning of a phrase ("not great").
Contractions - negations, but more complex ("wasn’t great").
Punctuation - increased intensity ("It’s great!!!").
Slang - variations of slang words such as "kinda", "sux", or "hella".
It's even able to understand acronyms ("lol") and emoji (❤).



In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
trainval["polarity_scores"] = trainval.phrase.map(lambda phrase : sid.polarity_scores(phrase))
test["polarity_scores"] = test.phrase.map(lambda phrase : sid.polarity_scores(phrase))
test.head()

In [ ]:
trainval["compound"] = trainval["polarity_scores"].map(lambda score_dict : score_dict["compound"])
test["compound"] = test["polarity_scores"].map(lambda score_dict : score_dict["compound"])

In [ ]:
trainval["prediction"] = trainval["compound"].map(lambda c: 1 if c >0 else 0 if c == 0 else -1)
test["prediction"] = test["compound"].map(lambda c: 1 if c >0 else 0 if c == 0 else -1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

print("VADER Sentiment Analysis Model")
print("TrainingValidation Data")
print(classification_report(trainval.label,trainval.prediction))
print("Test Data")
print(classification_report(test.label,test.prediction))

In [ ]:
print("TrainingValidation Data")
print(confusion_matrix(trainval.label,trainval.prediction))
print("Test Data")
print(confusion_matrix(test.label,test.prediction))

In [ ]:
# Examine wrong class -1
trainval.loc[(trainval.label == -1) & (trainval.prediction != -1)].phrase.values

##### Update Lexicon Dictionary (Round 1)

In [ ]:
new_food = {
    "tender" : 4,
    "fresh" : 4,
    "soggy" : -4,
    "jelat" : -4,
    "oily" : -4,
    "overcooked" :-4,
    "dry" : -2,
    "disappointed" : -4  
}

new_time = {
    "long queue" : -4,
    "queue" : -4,
    "wait" : -2,
    "slow" : -4,
    "crowd" : -4
}

new_price = {
    "pricey" : -4,
    "expensive" : -4,
    "cheap" : 4,
    "worth" : 4,
    "overpriced" : -4,
    "not worth" : -4,
    "value for money" : 4
    
}

new_portion = {
    "small" : -4,
    "large" : 4,
    "generous" : 4,
}

In [ ]:
sid.lexicon.update(new_food)
sid.lexicon.update(new_time)
sid.lexicon.update(new_price)
sid.lexicon.update(new_portion)

In [ ]:
trainval["polarity_scores1"] = trainval.phrase.map(lambda phrase : sid.polarity_scores(phrase))
test["polarity_scores1"] = test.phrase.map(lambda phrase : sid.polarity_scores(phrase))
trainval["compound1"] = trainval["polarity_scores1"].map(lambda score_dict : score_dict["compound"])
test["compound1"] = test["polarity_scores1"].map(lambda score_dict : score_dict["compound"])
trainval["prediction1"] = trainval["compound1"].map(lambda c: 1 if c >0 else 0 if c == 0 else -1)
test["prediction1"] = test["compound1"].map(lambda c: 1 if c >0 else 0 if c == 0 else -1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

print("VADER Sentiment Analysis Model (Tuned with new words)")
print("TrainingValidation Data")
print(classification_report(trainval.label,trainval.prediction1))
print("Test Data")
print(classification_report(test.label,test.prediction1))

In [ ]:
# Examine wrong class 1
trainval.loc[(trainval.label == 1) & (trainval.prediction1 != 1)].phrase.values

##### Update Lexicon Dictionary (Round 2)

In [ ]:
new_food = {
    "tender" : 2,
    "fresh" : 2,
    "soggy" : -2,
    "jelat" : -2,
    "oily" : -2,
    "overcooked" :-2,
    "dry" : -2,
    "disappointed" : -2,
    "cravings satisfied" : 2,
    "crispy" : 2,
    "sinful" : 2,
    "tough" : -2,
    "cold" : -2
}

new_time = {
    "long queue" : -2,
    "queue" : -2,
    "wait" : -2,
    "slow" : -2,
    "crowd" : -2,
    "crowded" : -2,
    "no waiting time" : 2,
    "fast" : 2,
}

new_price = {
    "pricey" : -2,
    "expensive" : -2,
    "cheap" : 2,
    "worth" : 2,
    "overpriced" : -2,
    "not worth" : -2,
    "value for money" : 2,
    "reasonable" : 2,
    "reasonably" : 2,
    "affordable" : 2,
    "steal" : 2
    
}

new_portion = {
    "small" : -2,
    "large" : 2,
    "generous" : 2,
    "sufficient" : 1,
    "enough" : 1
}

In [ ]:
sid.lexicon.update(new_food)
sid.lexicon.update(new_time)
sid.lexicon.update(new_price)
sid.lexicon.update(new_portion)

In [ ]:
trainval["polarity_scores2"] = trainval.phrase.map(lambda phrase : sid.polarity_scores(phrase))
test["polarity_scores2"] = test.phrase.map(lambda phrase : sid.polarity_scores(phrase))
trainval["compound2"] = trainval["polarity_scores2"].map(lambda score_dict : score_dict["compound"])
test["compound2"] = test["polarity_scores2"].map(lambda score_dict : score_dict["compound"])
trainval["prediction2"] = trainval["compound2"].map(lambda c: 1 if c >0 else 0 if c == 0 else -1)
test["prediction2"] = test["compound2"].map(lambda c: 1 if c >0 else 0 if c == 0 else -1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

print("VADER Sentiment Analysis Model (Tuned with new words 2)")
print("TrainingValidation Data")
print(classification_report(trainval.label,trainval.prediction2))
print("Test Data")
print(classification_report(test.label,test.prediction2))

In [ ]:
(34/383)*0.51 + (104/383)*0.58 + (245/383)*0.78